In [23]:
import os
import json
import requests
import base64
import pandas as pd
import time
from tqdm.auto import tqdm
import pickle

In [24]:
def call_refresh_api(refreshtoken: str):
    """
    idTokenをリフレッシュするメソッド。

    Parameters
    ----------
    refreshtoken : str
        refreshtoken。ログイン後の画面からご確認いただけます。

    Returns
    -------
    resjson : dict
        新しいidtokenが格納されたAPIレスポンス(json形式)
    """
    headers = {"accept": "application/json"}
    data = {"refresh-token": refreshtoken}

    response = requests.post(
        "https://api.jpx-jquants.com/refresh", headers=headers, data=json.dumps(data)
    )

    resjson = json.loads(response.text)
    return resjson


In [69]:
new_refresh_token = call_refresh_api(refreshtoke)

In [25]:
def call_jquants_api(params: dict, idtoken: str, apitype: str, code: str = None):
    """
    J-QuantsのAPIを試すメソッド。

    Parameters
    ----------
    params : dict
        リクエストパラメータ。
    idtoken : str
        idTokenはログイン後の画面からご確認いただけます。
    apitype: str
        APIの種類。"news", "prices", "lists"などがあります。
    code: str
        銘柄を指定するAPIの場合に設定します。

    Returns
    -------
    resjson : dict
        APIレスポンス(json形式)
    """
    datefrom = params.get("datefrom", None)
    dateto = params.get("dateto", None)
    date = params.get("date", None)
    includedetails = params.get("includedetails", "false")
    keyword = params.get("keyword", None)
    headline = params.get("headline", None)
    paramcode = params.get("code", None)
    nexttoken = params.get("nextToken", None)
    headers = {"accept": "application/json", "Authorization": idtoken}
    data = {
        "from": datefrom,
        "to": dateto,
        "includeDetails": includedetails,
        "nextToken": nexttoken,
        "date": date,
        "keyword": keyword,
        "headline": headline,
        "code": paramcode,
    }

    if code:
        code = "/" + code
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype + code,
            params=data,
            headers=headers,
        )
    else:
        r = requests.get(
            "https://api.jpx-jquants.com/" + apitype, params=data, headers=headers
        )
    if r.status_code != 200:
        print(f"Error : {r.status_code} Message : {r}")
        return
    resjson = json.loads(r.text)
    return resjson

# Set Token

In [26]:
idtk = "eyJraWQiOiJTelhkZGxsSEQrR0dyS0ZKRmdZZkNKdUFZdmxIOFdrVks5bXRcLzgrdFduTT0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI1YWYwN2U3YS1lMThkLTRhZTItYmRkZS0wMzllMzhjYTI2ZWEiLCJhdWQiOiI0bHYzZ2RuOXA1ZnF0cDcydmMzY3I5cG5xbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJldmVudF9pZCI6IjU4ZmY4ZmUxLWI0MjAtNDUxOS04ZDNhLWU4MDI2Yzk3ZmU0ZCIsInRva2VuX3VzZSI6ImlkIiwiYXV0aF90aW1lIjoxNjE2MjE2Mzc4LCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAuYXAtbm9ydGhlYXN0LTEuYW1hem9uYXdzLmNvbVwvYXAtbm9ydGhlYXN0LTFfM2o5eHZhYXlmIiwibmlja25hbWUiOiJTZWlzdWtlIFRhYmF5YXNoaSIsImNvZ25pdG86dXNlcm5hbWUiOiI1YWYwN2U3YS1lMThkLTRhZTItYmRkZS0wMzllMzhjYTI2ZWEiLCJleHAiOjE2MTYzNTczNjYsImlhdCI6MTYxNjM1Mzc2NiwiZW1haWwiOiJzdG1vZGUyMDAzQGdtYWlsLmNvbSJ9.ueDRBOKB28CoTTFxPSoPr5Cm9-cCOvH3sKeGOcPLHMTziHyXNV1LH2VnuWPolancF9aO1m55aaRvKyiZmvhlRhVPHnuqNWqjVLjKACMtrLC_vTqP6iOcmV0Ki14Ml9sDagPbpCK1I__rOhApePbaEpFhlR7qo-6CKtHkITuJQkTt6qJK_Fsu_rcD8QHH3f-ZYLzoEas6cPZLtE3PIws92dOxPg7-Pdjlqt-N3f1zGocq7vOmqMqYNNqw3ge_zNKPtmQUXs5q2f08mPSiqqnd-Nfov1sSOg0TJlD0w_7ODFliGVep0tfiY3J3MGV9AFu3tlGeSuH7KvsXh6RS0XKwXQ"

# Stock List

In [274]:
stock_list = pd.DataFrame()

is_next = True

paramdict = {}
paramdict["includedetails"] = "True"

while is_next:
    #time.sleep(10)
    response = call_jquants_api(paramdict, idtk, "lists")
    stock_list = stock_list.append(pd.DataFrame(response['list']))

    # 日別データが一度で読み取れなかった場合
    try:
        paramdict["nextToken"] = response['nextToken']
    except KeyError:
        is_next = False

In [276]:
model_path = os.path.join(os.path.dirname("__file__"), "../../get_data")
# tag::save_model[]
# モデル保存先ディレクトリを作成
os.makedirs(model_path, exist_ok=True)
with open(os.path.join(model_path, "stock_list.pkl"), "wb") as f:
    # モデルをpickle形式で保存
    pickle.dump(stock_list, f)

In [277]:
stock_list.to_csv(os.path.join(model_path, "stock_list.csv"), index=None)

# 取得する日付のリストを作成する

In [31]:
# 使用するモジュールのインポート
from datetime import datetime as dt
from datetime import timedelta

# 日付条件の設定
strdt = dt.strptime("2021-01-01", '%Y-%m-%d')  # 開始日
enddt = dt.strptime("2021-03-20", '%Y-%m-%d')  # 終了日

# 日付差の日数を算出（リストに最終日も含めたいので、＋１しています）
days_num = (enddt - strdt).days + 1  # （参考）括弧の部分はtimedelta型のオブジェクトになります
datelist = [strdt + timedelta(days=x) for x in range(days_num)]

# 日付毎に株価情報を取得する

In [235]:
# 株価情報取得
price_list = pd.DataFrame()

for date in tqdm(datelist):
    is_next = True
    print(date)
    paramdict = {}
    paramdict["date"] = date.strftime("%Y-%m-%d")
    paramdict["includedetails"] = "True"
    
    while is_next:
        #time.sleep(10)
        response = call_jquants_api(paramdict, idtk, "prices")
        # 休場の場合は、空のリストで返却される
        if response['prices'] == []:
            is_next = False
        else:
            price_list = price_list.append(pd.DataFrame(response['prices']))
            
            # 日別データが一度で読み取れなかった場合
            try:
                paramdict["nextToken"] = response['nextToken']
            except KeyError:
                is_next = False

2021-01-01 00:00:00
2021-01-02 00:00:00
2021-01-03 00:00:00
2021-01-04 00:00:00
2021-01-05 00:00:00
2021-01-06 00:00:00
2021-01-07 00:00:00
2021-01-08 00:00:00
2021-01-09 00:00:00
2021-01-10 00:00:00
2021-01-11 00:00:00
2021-01-12 00:00:00
2021-01-13 00:00:00
2021-01-14 00:00:00
2021-01-15 00:00:00
2021-01-16 00:00:00
2021-01-17 00:00:00
2021-01-18 00:00:00
2021-01-19 00:00:00
2021-01-20 00:00:00
2021-01-21 00:00:00
2021-01-22 00:00:00
2021-01-23 00:00:00
2021-01-24 00:00:00
2021-01-25 00:00:00
2021-01-26 00:00:00
2021-01-27 00:00:00
2021-01-28 00:00:00
2021-01-29 00:00:00
2021-01-30 00:00:00
2021-01-31 00:00:00
2021-02-01 00:00:00
2021-02-02 00:00:00
2021-02-03 00:00:00
2021-02-04 00:00:00
2021-02-05 00:00:00
2021-02-06 00:00:00
2021-02-07 00:00:00
2021-02-08 00:00:00
2021-02-09 00:00:00
2021-02-10 00:00:00
2021-02-11 00:00:00
2021-02-12 00:00:00
2021-02-13 00:00:00
2021-02-14 00:00:00
2021-02-15 00:00:00
2021-02-16 00:00:00
2021-02-17 00:00:00
2021-02-18 00:00:00
2021-02-19 00:00:00


In [238]:
price_list = price_list.sort_values(['EndOfDayQuote Date', 'Local Code'])

In [243]:
price_list.to_csv("price_list.csv", index=None)

In [245]:
model_path = os.path.join(os.path.dirname("__file__"), "../../get_data")
# tag::save_model[]
# モデル保存先ディレクトリを作成
os.makedirs(model_path, exist_ok=True)
with open(os.path.join(model_path, "stock_price.pkl"), "wb") as f:
    # モデルをpickle形式で保存
    pickle.dump(price_list, f)

# Fundamentals

In [250]:
idtk = "eyJraWQiOiJTelhkZGxsSEQrR0dyS0ZKRmdZZkNKdUFZdmxIOFdrVks5bXRcLzgrdFduTT0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI1YWYwN2U3YS1lMThkLTRhZTItYmRkZS0wMzllMzhjYTI2ZWEiLCJhdWQiOiI0bHYzZ2RuOXA1ZnF0cDcydmMzY3I5cG5xbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJldmVudF9pZCI6IjU4ZmY4ZmUxLWI0MjAtNDUxOS04ZDNhLWU4MDI2Yzk3ZmU0ZCIsInRva2VuX3VzZSI6ImlkIiwiYXV0aF90aW1lIjoxNjE2MjE2Mzc4LCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAuYXAtbm9ydGhlYXN0LTEuYW1hem9uYXdzLmNvbVwvYXAtbm9ydGhlYXN0LTFfM2o5eHZhYXlmIiwibmlja25hbWUiOiJTZWlzdWtlIFRhYmF5YXNoaSIsImNvZ25pdG86dXNlcm5hbWUiOiI1YWYwN2U3YS1lMThkLTRhZTItYmRkZS0wMzllMzhjYTI2ZWEiLCJleHAiOjE2MTYyNzg0NjgsImlhdCI6MTYxNjI3NDg2OCwiZW1haWwiOiJzdG1vZGUyMDAzQGdtYWlsLmNvbSJ9.RXGGFh2Iq6MPk7XgXNktrI7T8aQ5w7_q_GH8vNzUJdH-SUE3_pgBAJwimlZ2j2Bxq0GPg7UZuhXTxa85OJwHHDeU_HLxMLdiwGTThbRTEPM8j0bgAKLkk-cVt0qVO-OcAbQUV1C0lv3-e1j8AwWmZTL79Yb5HFfvgpMvHCOyg6u4gqIlhsZBMfbsCMnhjldrNx1e79MvCq34tpOKfl4h6ois8AeMbRxaIleZwlmDUAHk1TMU_7neINkPleD2r0EzXi4cbg5Oe-_aBDW3vjr_3UX1N7xY5D1vm28YQmtguOxRps6GNCmdNGV6rtsLatsaHPDMyR98pzJEfShEdqU1IA"

In [255]:
# ファンダメンタル情報取得
stock_fins = pd.DataFrame()

for date in tqdm(datelist):
    is_next = True
    print(date)
    paramdict = {}
    paramdict["date"] = date.strftime("%Y-%m-%d")
    paramdict["includedetails"] = "True"
    
    while is_next:
        #time.sleep(10)
        response = call_jquants_api(paramdict, idtk, "stockfins")
        # 休場の場合は、空のリストで返却される
        if response['stockfin'] == []:
            is_next = False
        else:
            stock_fins = stock_fins.append(pd.DataFrame(response['stockfin']))
            
            # 日別データが一度で読み取れなかった場合
            try:
                paramdict["nextToken"] = response['nextToken']
            except KeyError:
                is_next = False

2021-01-01 00:00:00
2021-01-02 00:00:00
2021-01-03 00:00:00
2021-01-04 00:00:00
2021-01-05 00:00:00
2021-01-06 00:00:00
2021-01-07 00:00:00
2021-01-08 00:00:00
2021-01-09 00:00:00
2021-01-10 00:00:00
2021-01-11 00:00:00
2021-01-12 00:00:00
2021-01-13 00:00:00
2021-01-14 00:00:00
2021-01-15 00:00:00
2021-01-16 00:00:00
2021-01-17 00:00:00
2021-01-18 00:00:00
2021-01-19 00:00:00
2021-01-20 00:00:00
2021-01-21 00:00:00
2021-01-22 00:00:00
2021-01-23 00:00:00
2021-01-24 00:00:00
2021-01-25 00:00:00
2021-01-26 00:00:00
2021-01-27 00:00:00
2021-01-28 00:00:00
2021-01-29 00:00:00
2021-01-30 00:00:00
2021-01-31 00:00:00
2021-02-01 00:00:00
2021-02-02 00:00:00
2021-02-03 00:00:00
2021-02-04 00:00:00
2021-02-05 00:00:00
2021-02-06 00:00:00
2021-02-07 00:00:00
2021-02-08 00:00:00
2021-02-09 00:00:00
2021-02-10 00:00:00
2021-02-11 00:00:00
2021-02-12 00:00:00
2021-02-13 00:00:00
2021-02-14 00:00:00
2021-02-15 00:00:00
2021-02-16 00:00:00
2021-02-17 00:00:00
2021-02-18 00:00:00
2021-02-19 00:00:00


In [257]:
stock_fins

,Forecast_Dividend AnnualDividendPerShare,Forecast_Dividend FiscalPeriodEnd,Forecast_Dividend FiscalYear,Forecast_Dividend ModifyDate,Forecast_Dividend QuarterlyDividendPerShare,Forecast_Dividend RecordDate,Forecast_Dividend ReportType,Forecast_FinancialStatement AccountingStandard,Forecast_FinancialStatement ChangeOfFiscalYearEnd,Forecast_FinancialStatement CompanyType,...,Result_FinancialStatement FiscalYear,Result_FinancialStatement ModifyDate,Result_FinancialStatement NetAssets,Result_FinancialStatement NetIncome,Result_FinancialStatement NetSales,Result_FinancialStatement OperatingIncome,Result_FinancialStatement OrdinaryIncome,Result_FinancialStatement ReportType,Result_FinancialStatement TotalAssets,base_date
0,,,,,,,,,,,...,2021.0,2021/01/04,19902.0,-1249.0,16673,-1622,-1231,Q3,24760.0,2021/01/04
1,0,2020/12,2020,2021/01/04,0,2020/12/31,Annual,NonConsolidated,False,GB,...,2020.0,2020/11/12,17405.0,701.0,3832,998,940,Q3,19026.0,2021/01/04
0,28,2021/05,2021,2021/01/05,17,2021/05/31,Annual,ConsolidatedJP,False,GB,...,2021.0,2021/01/05,20640.0,223.0,26181,246,300,Q2,36846.0,2021/01/05
1,6,2021/02,2021,2021/01/05,3,2021/02/28,Annual,ConsolidatedJP,False,GB,...,2021.0,2021/01/05,13490.0,443.0,11055,789,805,Q3,20912.0,2021/01/05
2,,2021/02,2021,2021/01/05,0,2021/02/28,Q2,NonConsolidated,False,GB,...,2021.0,2021/01/05,5113.0,100.0,1994,133,147,Q1,8675.0,2021/01/05
3,53,2021/02,2021,2021/01/05,53,2021/02/28,Annual,ConsolidatedJP,False,GB,...,2021.0,2021/01/05,129526.0,4527.0,151294,6660,7008,Q3,164084.0,2021/01/05
4,0,2021/02,2021,2021/01/05,0,2021/02/28,Annual,ConsolidatedJP,False,GB,...,2021.0,2021/01/05,2202.0,-726.0,4053,-401,-384,Q3,5965.0,2021/01/05
5,110,2021/03,2021,2021/01/05,55,2021/03/31,Annual,ConsolidatedIFRS,False,GB,...,2021.0,2020/11/06,387716.0,21973.0,241131,31840,32148,Q2,608818.0,2021/01/05
6,,2021/02,2021,2021/01/05,10,2021/02/28,Q2,ConsolidatedJP,False,GB,...,2021.0,2021/01/05,10546.0,375.0,4637,558,557,Q1,19393.0,2021/01/05
7,13.5,2020/11,2020,2021/01/05,13.5,2020/11/30,Annual,NonConsolidated,False,GB,...,2020.0,2020/10/14,1042.0,148.0,1652,215,216,Q3,1271.0,2021/01/05


In [258]:
model_path = os.path.join(os.path.dirname("__file__"), "../../get_data")
# tag::save_model[]
# モデル保存先ディレクトリを作成
os.makedirs(model_path, exist_ok=True)
with open(os.path.join(model_path, "stock_fin.pkl"), "wb") as f:
    # モデルをpickle形式で保存
    pickle.dump(stock_fins, f)

In [260]:
stock_fins.to_csv(os.path.join(model_path, "stock_fin.csv"), index=None)

# Stock Labels

In [32]:
# 株価上昇率情報取得
stock_labels = pd.DataFrame()

for date in tqdm(datelist):
    is_next = True
    print(date)
    paramdict = {}
    paramdict["date"] = date.strftime("%Y-%m-%d")
    paramdict["includedetails"] = "True"
    
    while is_next:
        #time.sleep(10)
        response = call_jquants_api(paramdict, idtk, "stocklabels")
        # 休場の場合は、空のリストで返却される
        if response['labels'] == []:
            is_next = False
        else:
            stock_labels = stock_labels.append(pd.DataFrame(response['labels']))
            
            # 日別データが一度で読み取れなかった場合
            try:
                paramdict["nextToken"] = response['nextToken']
            except KeyError:
                is_next = False

2021-01-01 00:00:00
2021-01-02 00:00:00
2021-01-03 00:00:00
2021-01-04 00:00:00
2021-01-05 00:00:00
2021-01-06 00:00:00
2021-01-07 00:00:00
2021-01-08 00:00:00
2021-01-09 00:00:00
2021-01-10 00:00:00
2021-01-11 00:00:00
2021-01-12 00:00:00
2021-01-13 00:00:00
2021-01-14 00:00:00
2021-01-15 00:00:00
2021-01-16 00:00:00
2021-01-17 00:00:00
2021-01-18 00:00:00
2021-01-19 00:00:00
2021-01-20 00:00:00
2021-01-21 00:00:00
2021-01-22 00:00:00
2021-01-23 00:00:00
2021-01-24 00:00:00
2021-01-25 00:00:00
2021-01-26 00:00:00
2021-01-27 00:00:00
2021-01-28 00:00:00
2021-01-29 00:00:00
2021-01-30 00:00:00
2021-01-31 00:00:00
2021-02-01 00:00:00
2021-02-02 00:00:00
2021-02-03 00:00:00
2021-02-04 00:00:00
2021-02-05 00:00:00
2021-02-06 00:00:00
2021-02-07 00:00:00
2021-02-08 00:00:00
2021-02-09 00:00:00
2021-02-10 00:00:00
2021-02-11 00:00:00
2021-02-12 00:00:00
2021-02-13 00:00:00
2021-02-14 00:00:00
2021-02-15 00:00:00
2021-02-16 00:00:00
2021-02-17 00:00:00
2021-02-18 00:00:00
2021-02-19 00:00:00


In [33]:
stock_labels

,Local Code,base_date,label_date_10,label_date_20,label_date_5,label_high_10,label_high_20,label_high_5,label_low_10,label_low_20,label_low_5
0,1301,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0.0661,0.0661,0.06102,-0.00508,-0.00508,-0.00508
1,1332,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0.06176,0.06888,0.05463,-0.0095,-0.0095,-0.0095
2,1333,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0.04768,0.09491,0.03588,-0.01317,-0.01317,-0.01317
3,1352,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0.03327,0.03327,0.03327,-0.00504,-0.00907,-0.00504
4,1375,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0.0914,0.10514,0.06571,-0.01075,-0.01075,-0.01075
5,1376,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0.02267,0.02267,0.0063,-0.03023,-0.03526,-0.0296
6,1377,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0.08427,0.08427,0.02949,-0.0014,-0.0014,-0.0014
7,1379,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0.02527,0.05055,0.02241,-0.00429,-0.00429,-0.00429
8,1380,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0.0083,0.01079,0.0083,-0.07137,-0.07137,-0.03154
9,1381,2021-01-04,2021-01-19,2021-02-02,2021-01-12,0,0.02722,0,-0.04441,-0.07593,-0.04155


In [34]:
model_path = os.path.join(os.path.dirname("__file__"), "../../get_data")
# tag::save_model[]
# モデル保存先ディレクトリを作成
os.makedirs(model_path, exist_ok=True)
with open(os.path.join(model_path, "stock_labels.pkl"), "wb") as f:
    # モデルをpickle形式で保存
    pickle.dump(stock_labels, f)

In [35]:
stock_labels.to_csv(os.path.join(model_path, "stock_labels.csv"), index=None)

In [38]:
# Codeを指定する場合
paramdict = {}
paramdict["from"] = "2021-02-21"
paramdict["to"] = "2021-02-28"
paramdict["includedetails"] = "true"
responce = call_jquants_api(paramdict, idtk, "stocklabels", "9984")

In [39]:
responce

{'labels': [{'label_low_10': -0.1717,
   'label_low_20': -0.24245,
   'label_low_5': -0.07792,
   'label_high_20': 0.00884,
   'base_date': '2016-01-04',
   'label_high_10': 0.00884,
   'label_date_5': '2016-01-12',
   'label_date_10': '2016-01-19',
   'label_high_5': 0.00884,
   'label_date_20': '2016-02-02',
   'Local Code': '9984'},
  {'label_low_10': -0.21595,
   'label_low_20': -0.2394,
   'label_low_5': -0.07422,
   'label_high_20': 0.00519,
   'base_date': '2016-01-05',
   'label_high_10': 0.00519,
   'label_date_5': '2016-01-13',
   'label_date_10': '2016-01-20',
   'label_high_5': 0.00519,
   'label_date_20': '2016-02-03',
   'Local Code': '9984'},
  {'label_low_10': -0.22907,
   'label_low_20': -0.22907,
   'label_low_5': -0.08473,
   'label_high_20': 0.0034,
   'base_date': '2016-01-06',
   'label_high_10': 0.0034,
   'label_date_5': '2016-01-14',
   'label_date_10': '2016-01-21',
   'label_high_5': 0.0034,
   'label_date_20': '2016-02-04',
   'Local Code': '9984'},
  {'labe